In [0]:
df = df.withColumn("ingestion_date", current_timestamp())

df.write.format("delta").mode("overwrite").saveAsTable("sncf_analytics.raw.salesdata")

In [0]:
from pyspark.sql.functions import col, to_timestamp, when

df = spark.table("sncf_trafic.raw.salesdata")

df_transformed = (
    df
    .withColumn("DepartureTime", to_timestamp(col("DepartureTime")))
    .withColumn("DelayMinutes", col("DelayMinutes").cast("int"))
    .withColumn("Status", when(col("DelayMinutes") > 10, "Late").otherwise("On Time"))
)

df_transformed.write.format("delta").mode("overwrite").saveAsTable("sncf_trafic.curated.train_status")


In [0]:
data = """TrainID,TrainType,DepartureStation,ArrivalStation,DepartureTime,ArrivalTime,DelayMinutes
TGV001,TGV,Paris,Lyon,2024-10-30 06:45:00,2024-10-30 09:05:00,0
TGV002,TGV,Paris,Marseille,2024-10-30 07:00:00,2024-10-30 10:00:00,8
TGV003,TGV,Lyon,Paris,2024-10-30 07:15:00,2024-10-30 09:35:00,3
TGV004,TGV,Marseille,Paris,2024-10-30 08:00:00,2024-10-30 11:00:00,12
TGV005,TGV,Paris,Bordeaux,2024-10-30 09:00:00,2024-10-30 11:20:00,0
TER101,TER,Lille,Paris,2024-10-30 07:30:00,2024-10-30 09:45:00,15
TER102,TER,Reims,Paris,2024-10-30 08:10:00,2024-10-30 09:50:00,2
TER103,TER,Paris,Chartres,2024-10-30 10:00:00,2024-10-30 11:05:00,0
TER104,TER,Orléans,Paris,2024-10-30 10:30:00,2024-10-30 12:00:00,20
TER105,TER,Paris,Lille,2024-10-30 11:00:00,2024-10-30 13:10:00,5
IC201,Intercités,Toulouse,Bordeaux,2024-10-30 06:30:00,2024-10-30 08:45:00,0
IC202,Intercités,Bordeaux,Nantes,2024-10-30 09:00:00,2024-10-30 12:15:00,18
IC203,Intercités,Nantes,Bordeaux,2024-10-30 13:00:00,2024-10-30 16:10:00,10
IC204,Intercités,Bordeaux,Toulouse,2024-10-30 16:30:00,2024-10-30 18:45:00,0
TGV006,TGV,Paris,Lyon,2024-10-30 12:00:00,2024-10-30 14:20:00,25
TGV007,TGV,Lyon,Paris,2024-10-30 15:00:00,2024-10-30 17:15:00,0
TER106,TER,Paris,Reims,2024-10-30 16:00:00,2024-10-30 17:30:00,4
TER107,TER,Paris,Orléans,2024-10-30 17:30:00,2024-10-30 18:50:00,0
TGV008,TGV,Marseille,Lyon,2024-10-30 18:00:00,2024-10-30 19:45:00,10
TGV009,TGV,Paris,Strasbourg,2024-10-30 19:00:00,2024-10-30 21:15:00,7
"""

path = "abfss://root@dbstorageqf6i3byf7qjb4.dfs.core.windows.net/3073971834093534/user/hive/warehouse/Dev/Raw/train_schedule.csv"
dbutils.fs.put(
    path,
    data,
    overwrite=True
)

print(f"✅ Dataset créé : {path}")

In [0]:
try:
    dbutils.fs.ls("abfss://root@dbstorageqf6i3byf7qjb4.dfs.core.windows.net/3073971834093534/user/hive/warehouse/Dev/Raw/")
    print("You have access to this path.")
except Exception as e:
    print(f"No access or error: {e}")

In [0]:
file_path = "abfss://root@dbstorageqf6i3byf7qjb4.dfs.core.windows.net/3073971834093534/user/hive/warehouse/Dev/Raw/train_schedule.csv"

df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)
display(df)

In [0]:
data = """TrainID,TrainType,DepartureStation,ArrivalStation,DepartureTime,ArrivalTime,DelayMinutes
TGV001,TGV,Paris,Lyon,2024-10-30 06:45:00,2024-10-30 09:05:00,0
TGV002,TGV,Paris,Marseille,2024-10-30 07:00:00,2024-10-30 10:00:00,8
TGV003,TGV,Lyon,Paris,2024-10-30 07:15:00,2024-10-30 09:35:00,3
TGV004,TGV,Marseille,Paris,2024-10-30 08:00:00,2024-10-30 11:00:00,12
TGV005,TGV,Paris,Bordeaux,2024-10-30 09:00:00,2024-10-30 11:20:00,0
TER101,TER,Lille,Paris,2024-10-30 07:30:00,2024-10-30 09:45:00,15
TER102,TER,Reims,Paris,2024-10-30 08:10:00,2024-10-30 09:50:00,2
TER103,TER,Paris,Chartres,2024-10-30 10:00:00,2024-10-30 11:05:00,0
TER104,TER,Orléans,Paris,2024-10-30 10:30:00,2024-10-30 12:00:00,20
TER105,TER,Paris,Lille,2024-10-30 11:00:00,2024-10-30 13:10:00,5
IC201,Intercités,Toulouse,Bordeaux,2024-10-30 06:30:00,2024-10-30 08:45:00,0
IC202,Intercités,Bordeaux,Nantes,2024-10-30 09:00:00,2024-10-30 12:15:00,18
IC203,Intercités,Nantes,Bordeaux,2024-10-30 13:00:00,2024-10-30 16:10:00,10
IC204,Intercités,Bordeaux,Toulouse,2024-10-30 16:30:00,2024-10-30 18:45:00,0
TGV006,TGV,Paris,Lyon,2024-10-30 12:00:00,2024-10-30 14:20:00,25
TGV007,TGV,Lyon,Paris,2024-10-30 15:00:00,2024-10-30 17:15:00,0
TER106,TER,Paris,Reims,2024-10-30 16:00:00,2024-10-30 17:30:00,4
TER107,TER,Paris,Orléans,2024-10-30 17:30:00,2024-10-30 18:50:00,0
"""

# Création du fichier CSV dans DBFS
path = "/dbfs/lab1/train_schedule.csv"
dbutils.fs.put("dbfs:/lab1/train_schedule.csv", data, overwrite=True)

print(f"✅ Fichier créé : {path}")

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("sncf_trafic_dev.raw.train_schedule")


In [0]:
# -------------------------------------------------------------------------
# Import des fonctions nécessaires
# -------------------------------------------------------------------------
from pyspark.sql.functions import col, to_timestamp, when

# -------------------------------------------------------------------------
# Lecture de la table brute (RAW Layer)
# -------------------------------------------------------------------------
df = spark.table("sncf_trafic_dev.raw.train_schedule")

# -------------------------------------------------------------------------
# Transformation et typage des colonnes temporelles
# -------------------------------------------------------------------------
df_transformed = ( 
    df
    # Convertit la colonne DepartureTime en type timestamp
    .withColumn("DepartureTime", to_timestamp(col("DepartureTime")))
    
    # Convertit la colonne ArrivalTime en type timestamp
    .withColumn("ArrivalTime", to_timestamp(col("ArrivalTime")))

    # Convertit DelayMinutes en entier (pour les calculs)
    .withColumn("DelayMinutes", col("DelayMinutes").cast("int"))

    # Crée une nouvelle colonne "Status" selon le retard
    .withColumn("Status", when(col("DelayMinutes") > 10, "Late").otherwise("On Time"))
)

# -------------------------------------------------------------------------
# Écriture des données transformées dans la couche CURATED (Delta Lake)
# -------------------------------------------------------------------------
df_transformed.write.format("delta").mode("overwrite").saveAsTable("sncf_trafic_dev.curated.train_status")

display(df_transformed)

In [0]:
%sql
-- Liste des tables disponibles
SHOW TABLES IN sncf_analytics.curated;

-- Vérifier le contenu
SELECT TrainID, TrainType, DelayMinutes, Status FROM sncf_trafic_dev.curated.train_status;

-- Comptage et analyse de ponctualité
SELECT Status, COUNT(*) AS Nb_Trains
FROM sncf_trafic_dev.curated.train_status
GROUP BY Status;


In [0]:

# -------------------------------------------------------------------------
# Import des fonctions nécessaires
# -------------------------------------------------------------------------
from pyspark.sql.functions import col, to_timestamp, when, round

# -------------------------------------------------------------------------
# Lecture de la table RAW (importée depuis le CSV)
# -------------------------------------------------------------------------
df = spark.table("sncf_trafic_dev.raw.train_schedule")

# -------------------------------------------------------------------------
# Transformation et enrichissement
# -------------------------------------------------------------------------
df_transformed = (
    df
    # Typage des colonnes temporelles
    .withColumn("DepartureTime", to_timestamp(col("DepartureTime")))
    .withColumn("ArrivalTime", to_timestamp(col("ArrivalTime")))
    .withColumn("DelayMinutes", col("DelayMinutes").cast("int"))

    # Calcul du statut de ponctualité
    .withColumn("Status", when(col("DelayMinutes") > 10, "Late").otherwise("On Time"))

    # Calcul de la durée de trajet (ArrivalTime - DepartureTime)
    .withColumn("TravelDurationMinutes",
                round((col("ArrivalTime").cast("long") - col("DepartureTime").cast("long")) / 60, 2))
)

# -------------------------------------------------------------------------
# Écriture dans la table CURATED (Delta Lake + Unity Catalog)
# -------------------------------------------------------------------------
df_transformed.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("sncf_trafic_dev.curated.train_status")

print("✅ Table sncf_analytics.curated.train_status mise à jour avec la durée du trajet (TravelDurationMinutes).")


In [0]:
display(spark.sql("SELECT * FROM sncf_trafic_dev.curated.train_status LIMIT 10"))

In [0]:
sncf_trafic_dev.raw.train_schedulesncf_trafic_dev.raw.train_schedule%sql
-- Lister les tables disponibles dans la couche CURATED
SHOW TABLES IN sncf_analytics.curated;

-- Vérifier le contenu principal
SELECT TrainID, TrainType, DepartureStation, ArrivalStation,
       DepartureTime, ArrivalTime, DelayMinutes, Status, TravelDurationMinutes
FROM sncf_trafic_dev.curated.train_status
ORDER BY DepartureTime;

-- Vérifier les moyennes de durée et retards
SELECT TrainType,
       ROUND(AVG(DelayMinutes),2) AS AvgDelay,
       ROUND(AVG(TravelDurationMinutes),2) AS AvgDuration
FROM sncf_trafic_dev.curated.train_status
GROUP BY TrainType;


Databricks visualization. Run in Databricks to view.

In [0]:
display(spark.table("sncf_trafic_dev.curated.train_status"))

In [0]:
from pyspark.sql.functions import avg, max, min, count, sum

# Moyenne du retard par type de train
agg_avg_delay = df.groupBy("TrainType").agg(avg("DelayMinutes").alias("AvgDelay"))
display(agg_avg_delay)

# Nombre de trajets par station de départ
agg_count_departure = df.groupBy("DepartureStation").agg(count("*").alias("NbTrajets"))
display(agg_count_departure)

# Retard maximum par type de train
agg_max_delay = df.groupBy("TrainType").agg(max("DelayMinutes").alias("MaxDelay"))
display(agg_max_delay)

# Somme totale des retards par station d'arrivée
agg_sum_delay = df.groupBy("ArrivalStation").agg(sum("DelayMinutes").alias("TotalDelay"))
display(agg_sum_delay)

# Durée de trajet minimale et maximale par type de train (si TravelDurationMinutes existe)
if "TravelDurationMinutes" in df.columns:
    agg_duration = df.groupBy("TrainType").agg(
        min("TravelDurationMinutes").alias("MinDuration"),
        max("TravelDurationMinutes").alias("MaxDuration")
    )
    display(agg_duration)

In [0]:
# Visualisation 1 : Répartition des statuts de ponctualité
df = spark.table("sncf_trafic_dev.curated.train_status")
display(df.groupBy("Status").count())

# Visualisation 2 : Moyenne du retard par type de train
display(df.groupBy("TrainType").avg("DelayMinutes"))

# Visualisation 3 : Durée moyenne de trajet par type de train
if "TravelDurationMinutes" in df.columns:
    display(df.groupBy("TrainType").avg("TravelDurationMinutes"))

# Visualisation 4 : Nombre de trajets par station de départ
display(df.groupBy("DepartureStation").count())

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.